## Subtractive Cancellation

__MATH 420__ <br>
_Spring 2021_ <br>

Herbie is an online tool (https://herbie.uwplse.org/demo/) that attempts to modify a formula into a formula that is equivalent, but evaluates more accurately using floating point numbers. It outputs its suggestion in the C programming language, so you might need to understand a bit of the C language to use Herbie.

To test Herbie, let's try the expression $(1-\exp(x))/x$.  When $x \approx 0$ we have $1 - \exp(x) \approx 1 - 1$.  So we expect subtractive cancellation when $x$ is near zero. Actually, we have
$$
   \lim_{x \to 0} \frac{1 - \exp(x)}{x} = -1.
$$
So when we test this code for $x \approx 0$, the result should be close to $-1$. Here is what Herbie recommends:

In [ ]:
using Gadfly

In [ ]:
function herbie(x::Float64)
    if x <=  -0.0013583059356520257
          1.0 / (x / (1.0 - exp(x)))
    else
        -1.0 + (x * (-0.5 + (x * ((x * -0.041666666666666664) + -0.16666666666666666))))
    end
end   

For $x < -0.0013583059356520257$, Herbie recommends a formula that is algebraically equivalent to the original, but for other values of $x$, it recommends the third order Taylor polynomial centered at zero for $(1-\exp(x))/x$. It's likely, and we'll show that it's true, that for largish inputs, Herbie's recommendation is inaccurate.

First let's compare Herbie's recommendation with the original formula:

In [ ]:
function nonherbie(x::Float64)
    (1-exp(x))/x
end

For $x$ near zero, here is a graphical comparison:

In [ ]:
plot([herbie, nonherbie], -1.0e-7, 1.0e-7)

The non Herbie code has _spurious_ oscillations for $x$ near zero, but the Herbie code looks like a line. The wild oscillations are rubbish, making Herbie's suggestion far more accurate near zero.

But the following graph shows that the Herbie's suggestion is inaccurate away from $0$. This isn't surprising--we don't expect that the third degree Taylor polynomial centered at zero to be accurate far away from zero. I wouldn't trust Herbie's recommendations to write code for a self-driving car.

In [ ]:
plot([herbie, nonherbie], 1,5)

Computing $\exp(x) - 1$ is such a common need (financial calculations, for example), that microprocessors have a built-in way to evaluate it accurately, even when $x \approx 0$. In Julia, the function name is `expm1` (for exp minus one). Using `expm1`, we can write our function as

In [ ]:
function herbie_alt(x::Number)
    -expm1(x)/x
end

It looks like herbie_alt and herbie agree on the interval $[-10^{-7}, 10^{-7}]$

In [ ]:
plot([herbie, herbie_alt], -1.0e-7, 1.0e-7)

And away from zero, both `nonherbie` and `herbie_alt` agree.

In [ ]:
plot([nonherbie, herbie_alt], 1,5)

Our last example is from finance. The function `payment` determines the payment for a loan amount of $P$ for an annual interest rate or $r$ and requiring $n$ payments to pay off the loan. A simple implementation is

In [ ]:
function payment(P::Number, r::Number, n::Integer)
    P*r*(1+r)^n/((1+r)^n-1)
end

Let's use the `expm1` function to rewrite the denominator

In [ ]:
function payment2(P::Number, r::Number, n::Integer)
    w = expm1(n*log(1+r))    
    P*r*(w+1)/w
end

Does the fancy `expm1` function ever make a difference? Using `Float32` numbers and choosing $P = 10^6, r = 0.001$ and $n = 2$, we have

In [ ]:
P = 1.0f6;

In [ ]:
r = 0.001f0;

In [ ]:
n = 2;

In [ ]:
p1 = payment(P,r,n)

In [ ]:
p2 = payment2(P,r,n)

OK, the difference \$11.59 for a million dollar loan. 

In [ ]:
p1-p2

Switching to `Float64` numbers, the difference is essentially zero:

In [ ]:
P = convert(Float64,P)

In [ ]:
r = convert(Float64,r)

In [ ]:
p1 = payment(P,r,n)

In [ ]:
p2 = payment2(P,r,n)

In [ ]:
p1-p2